<a href="https://colab.research.google.com/github/rahiakela/transformers-research-and-practice/blob/main/natural-language-processing-with-transformers/01-hello-transformers/tour_of_transformer_applications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Tour of Transformer Applications

Text is everywhere around us and being able to understand and act on information we can find
in text is a crucial aspect in every company. Every NLP task starts with a piece of text, like the
following made-up customer feedback about a certain online order:

```python
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""
```

Depending on your application, this text could be a legal contract, a product description or
something else entirely.


##Setup

In [18]:
!pip -q install transformers[sentencepiece]

In [5]:
from transformers import pipeline
from transformers import set_seed

import pandas as pd

In [2]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

##Text Classification

Transformers has a layered API which allows you to interact with
the library at various levels of abstraction. 

We’ll start with the most high level
API pipelines, which abstract away all the steps needed to convert raw text into a set of predictions from a fine-tuned model.

In Transformers, we instantiate a pipeline by providing the name of the task we are
interested in:

In [ ]:
classifier = pipeline("sentiment-analysis")

Now that we have our pipeline, let’s generate some predictions! Each pipeline takes a string of
text (or a list of strings) as input and returns a list of predictions. Each prediction is a Python
dictionary, so we can use Pandas to display them nicely as a DataFrame:

In [9]:
outputs = classifier(text)
pd.DataFrame.from_records(outputs)

,label,score
0,NEGATIVE,0.901546


In this case the model is very confident that the text has a negative sentiment, which makes
sense given that we’re dealing with complaint from an irate Autobot!

##Named Entity Recognition

Predicting the sentiment of customer feedback is a good first step, but you often want to know
if the feedback was about a particular product or service. Names of products, places or people
are called named entities and detecting and extracting them from text is called named entity
recognition (NER). 

We can apply NER by spinning up the corresponding pipeline and feeding
our piece of text to it:

In [ ]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")

In [8]:
outputs = ner_tagger(text)
pd.DataFrame.from_records(outputs)

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556570,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


You can see that the pipeline detected all the entities and also assigned a category such as ORG
(organization), LOC (location) or PER (person) to them.

Here we used the
aggregation_strategy argument to group the words according to the model’s
predictions; for example “Optimus Prime” has two words but is assigned a single MISC
(miscellaneous) category. The scores tell us how confident the model was about the entity and
we can see that it was least confident about “Decepticons” and the first occurrence of
“Megatron”, both of which it failed to group as a single entity.

Extracting all the named entities is nice but sometimes we would like to ask more targeted
questions. This is where we can use question answering.

##Question Answering

In question answering we provide the model with a passage of text called the context, along
with a question whose answer we’d like to extract. The model then returns the span of text
corresponding to the answer. 

So let’s see what we get when we ask a specific question about the
text:

In [ ]:
reader = pipeline("question-answering")

In [12]:
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame.from_records([outputs])

,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


There are several flavors of question answering, but this particular kind
is called extractive question answering because the answer is extracted directly from the text.

With this approach you can read and extract relevant information quickly from a customer’s feedback.

But what if you get a mountain of long-winded complaints and you don’t have the
time to read them all? Let’s see if a summarization model can help!

##Summarization

The goal of text summarization is to take a long text as input and generate a short version with
all relevant facts. This is a much more complicated task than the previous ones since it requires the model to produce coherent text as output. 

In what should be a familiar pattern by now, we
can instantiate a summarization pipeline as follows:

In [ ]:
summarizer = pipeline("summarization")

In [14]:
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0]["summary_text"])

 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead.


This isn’t too bad! Although parts of the original text have been copy-pasted, the model was
able to correctly identify that “Bumblebee” (which appeared at the end) was the author of the
complaint, and has captured the essence of the problem.

But what happens when you get a feedback that is in a language you don’t
understand? You could use Google Translate or you can use your very own transformer to
translate it for you!

##Translation

Like summarization, translation is a task where the output consists of generated text. 

Let’s use
the translation pipeline to translate the English text to German:

In [ ]:
translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")

In [4]:
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]["translation_text"])

Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt. Anbei sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, bald von Ihnen zu hören. Aufrichtig, Bumblebee.


Again, the model has produced a very good translation that correctly uses the formal pronouns
like “Ihrem” and “Sie” in German! 

Here we’ve also shown how you can override the default
model in the pipeline to pick the best one for your application, and you can find models for
thousands of language pairs on the Hub.

##Text Generation

Let’s say you would like to write faster answers to customer feedback by having access to an
autocomplete function.

With a text generation model you can continue an input text as follows:

In [ ]:
set_seed(42)

generator = pipeline("text-generation")

In [8]:
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."

prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. I have received your order. I would apologize in advance. I feel bad for you, but if my previous order wasn't received late, I apologize as well. I regret not sending you a package in advance. Thank you for your patience. We appreciate you coming to our store in Germany.

Packing for the Optimus Prime
